<a href="https://colab.research.google.com/github/dyanni3/Blog/blob/master/collab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get Started with TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To run the Colab notebook:

1. Connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

For more examples and guides (including details for this program), see [Get Started with TensorFlow](https://www.tensorflow.org/get_started/).

Let's get started, import the TensorFlow library into your program:

In [0]:
import tensorflow as tf

Load and prepare the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. Convert the samples from integers to floating-point numbers:

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras` model by stacking layers. Select an optimizer and loss function used for training:

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Train and evaluate model:

In [0]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test)

You’ve now trained an image classifier with ~98% accuracy on this dataset. See [Get Started with TensorFlow](https://www.tensorflow.org/get_started/) to learn more.

In [0]:
from matplotlib import pyplot as plt
import numpy as np
examples = x_train[:512]

In [0]:
fig, axes = plt.subplots(ncols=10,nrows=1)
for j in range(6):
  axes[j].imshow(examples[j])
results = model.predict(examples);
print([np.argmax(results[i]) for i in range(6)])

Simplest possible autoencoder. It's a fully connected layer as the encoder and decoder

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

In [0]:
#This uses the functional API. It doesn't use sequential (easiest)
# because we need access to just the encoder and just the decoder 
# but it's not as complicated as model subclassing.

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
#784 == 28**2, so this will be correct for MNIST images

# this is our input placeholder
input_img = Input(shape=(784,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [0]:
#now build just the encoder model
encoder = Model(input_img,encoded)

#and just the decoder model
encoded_input = Input(shape = (encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input,decoder_layer(encoded_input))


In [0]:
#now compile the autoencoder, tell it what loss function to use, what optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

The way to train the autoencoder is to make the y_train data be exactly the x_train data. i.e. you want to do a reconstruction, so you want the output to equal the input.


In [0]:
import numpy as np
print(x_train.shape)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)

In [0]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)
fig,ax = plt.subplots(nrows=3,ncols=6)
for j in range(3):
  for i in range(6):
    #plot the input
    ax[0][i].imshow(x_test[i].reshape(28,28),cmap=plt.cm.gray)
    #plot the reconstructed image
    ax[1][i].imshow(decoded_imgs[i].reshape(28,28),cmap = plt.cm.gray)
    #plot the encoded image
    ax[2][i].imshow(encoded_imgs[i].reshape(8,4),cmap=plt.cm.viridis_r)

## Sparsity:

The encoded images do not look very sparse. For the purpose of explainable AI we don't only want a low reconstruction error, we also want the encoded units to represent *concepts*. In order for that to be the case it's best if a given input highly activates only few encoded units.

